In [1]:
#SQL Imports
import mysql.connector
#Pandas imports
import pandas as pd
from datetime import datetime

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor()

In [3]:
match_and_betting_data = pd.read_csv('Data-Table 1.csv')

In [4]:
data = match_and_betting_data.rename(columns=match_and_betting_data.iloc[0])[1:].reset_index(drop=True)
data.head()

,Date,Kick-off (local),Home Team,Away Team,Home Score,Away Score,Play Off Game?,Over Time?,Home Odds,Draw Odds,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,06-Oct-19,19:30,Sydney Roosters,Canberra Raiders,14,8,Y,NaN,\t1.35,\t18.92,...,34.5,1.91,1.85,1.91,1.91,1.91,1.91,1.91,1.91,NaN
1,28-Sep-19,19:50,Sydney Roosters,Melbourne Storm,14,6,Y,NaN,\t1.59,\t19.04,...,34.5,1.97,1.91,2.00,2.00,1.85,1.82,1.91,1.82,NaN
2,27-Sep-19,19:50,Canberra Raiders,South Sydney Rabbitohs,16,10,Y,NaN,\t1.39,\t20.38,...,38.5,1.91,1.91,1.95,1.91,1.91,1.91,1.91,1.91,NaN
3,21-Sep-19,19:50,Melbourne Storm,Parramatta Eels,32,0,Y,NaN,\t1.35,\t21.14,...,36.5,1.91,1.91,1.91,2.00,1.91,1.82,1.91,1.82,NaN
4,20-Sep-19,19:50,South Sydney Rabbitohs,Manly Sea Eagles,34,26,Y,NaN,\t1.41,\t20.94,...,37.5,1.97,1.85,2.00,1.85,1.85,1.91,1.91,1.97,NaN


In [5]:
data.columns

Index(['Date', 'Kick-off (local)', 'Home Team', 'Away Team', 'Home Score',
       'Away Score', 'Play Off Game?', 'Over Time?', 'Home Odds', 'Draw Odds',
       'Away Odds', 'Bookmakers Surveyed', 'Home Odds Open', 'Home Odds Min',
       'Home Odds Max', 'Home Odds Close', 'Away Odds Open', 'Away Odds Min',
       'Away Odds Max', 'Away Odds Close', 'Home Line Open', 'Home Line Min',
       'Home Line Max', 'Home Line Close', 'Away Line Open', 'Away Line Min',
       'Away Line Max', 'Away Line Close', 'Home Line Odds Open',
       'Home Line Odds Min', 'Home Line Odds Max', 'Home Line Odds Close',
       'Away Line Odds Open', 'Away Line Odds Min', 'Away Line Odds Max',
       'Away Line Odds Close', 'Total Score Open', 'Total Score Min',
       'Total Score Max', 'Total Score Close', 'Total Score Over Open',
       'Total Score Over Min', 'Total Score Over Max',
       'Total Score Over Close', 'Total Score Under Open',
       'Total Score Under Min', 'Total Score Under Max',
      

In [6]:
data = data[['Date', 'Kick-off (local)', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Play Off Game?',
            'Home Odds', 'Draw Odds', 'Away Odds']]
data.head()

,Date,Kick-off (local),Home Team,Away Team,Home Score,Away Score,Play Off Game?,Home Odds,Draw Odds,Away Odds
0,06-Oct-19,19:30,Sydney Roosters,Canberra Raiders,14,8,Y,\t1.35,\t18.92,\t3.45
1,28-Sep-19,19:50,Sydney Roosters,Melbourne Storm,14,6,Y,\t1.59,\t19.04,\t2.48
2,27-Sep-19,19:50,Canberra Raiders,South Sydney Rabbitohs,16,10,Y,\t1.39,\t20.38,\t3.21
3,21-Sep-19,19:50,Melbourne Storm,Parramatta Eels,32,0,Y,\t1.35,\t21.14,\t3.36
4,20-Sep-19,19:50,South Sydney Rabbitohs,Manly Sea Eagles,34,26,Y,\t1.41,\t20.94,\t3.09


## Clean Data to Insert into Database

Colums in database Matches table:
    date, round, home_team_id, home_score, away_team_id, away_score, winner, is_draw, stadium_id, weather, url

1. Only want date starting with 2013
2. Need a way to easily find round of each match so we can interpolate into the url

In [7]:
data = data.rename(columns={'Home Score': 'home_score', 'Away Score': 'away_score'})

In [8]:
team_query = pd.read_sql_query('''
    SELECT * 
    FROM Teams;
''', mydb)
teams_df = pd.DataFrame(team_query)
teams_df

,id,official_name,nickname,abbreviation,logo_image,url
0,1,Brisbane,Broncos,BRI,None,https://www.broncos.com.au
1,2,Canberra,Raiders,CBR,None,https://www.raiders.com.au
2,3,Canterbury-Bankstown,Bulldogs,CBY,None,https://www.bulldogs.com.au
3,4,Cronulla-Sutherland,Sharks,CRO,None,https://www.sharks.com.au
4,5,Gold Coast,Titans,GLD,None,https://www.titans.com.au
5,6,Manly Warringah,Sea Eagles,MAN,None,https://www.seaeagles.com.au
6,7,Melbourne,Storm,MEL,None,https://www.melbournestorm.com.au
7,8,Newcastle,Knights,NEW,None,https://www.newcastleknights.com.au
8,9,New Zealand,Warriors,WAR,None,https://www.warriors.kiwi
9,10,North Queensland,Cowboys,NQL,None,https://www.cowboys.com.au


In [9]:
print(list(data['Home Team'].unique()))
print(list(data['Away Team'].unique()))

['Sydney Roosters', 'Canberra Raiders', 'Melbourne Storm', 'South Sydney Rabbitohs', 'Parramatta Eels', 'Manly Sea Eagles', 'Penrith Panthers', 'Wests Tigers', 'Gold Coast Titans', 'Canterbury Bulldogs', 'St George Dragons', 'Cronulla Sharks', 'Newcastle Knights', 'Brisbane Broncos', 'New Zealand Warriors', 'North QLD Cowboys', 'Manly-Warringah Sea Eagles', 'Canterbury-Bankstown Bulldogs', 'Cronulla-Sutherland Sharks', 'North Queensland Cowboys', 'St. George Illawarra Dragons']
['Canberra Raiders', 'Melbourne Storm', 'South Sydney Rabbitohs', 'Parramatta Eels', 'Manly Sea Eagles', 'Brisbane Broncos', 'Cronulla Sharks', 'Newcastle Knights', 'St George Dragons', 'New Zealand Warriors', 'North QLD Cowboys', 'Sydney Roosters', 'Wests Tigers', 'Penrith Panthers', 'Gold Coast Titans', 'Canterbury Bulldogs', 'Manly-Warringah Sea Eagles', 'Cronulla-Sutherland Sharks', 'North Queensland Cowboys', 'Canterbury-Bankstown Bulldogs', 'St. George Illawarra Dragons']


In [10]:
for t in ['Home Team', 'Away Team']:
    data.loc[data[t] == 'North QLD Cowboys', t] = 'North Queensland Cowboys'
    data.loc[data[t] == 'Manly Sea Eagles', t] = 'Manly Warringah Sea Eagles'
    data.loc[data[t] == 'Canterbury Bulldogs', t] = 'Canterbury-Bankstown Bulldogs'
    data.loc[data[t] == 'St George Dragons', t] = 'St. George Illawarra Dragons'
    data.loc[data[t] == 'Manly-Warringah Sea Eagles', t] = 'Manly Warringah Sea Eagles'
    data.loc[data[t] == 'Cronulla Sharks', t] = 'Cronulla-Sutherland Sharks'


In [11]:
#Test to make sure rows were properly updated
print(len(list(data['Home Team'].unique())))
print(len(list(data['Away Team'].unique())))

16
16


In [12]:
#Create df with team name and id to join on main dataframe
team_name_query = pd.read_sql_query('''
    SELECT id,
        CONCAT(official_name, ' ', nickname) 'Home Team',
        CONCAT(official_name, ' ', nickname) 'Away Team'
    FROM Teams;
''', mydb)
team_name_df = pd.DataFrame(team_name_query)
home_team_name_df = team_name_df[['id', 'Home Team']]
away_team_name_df = team_name_df[['id', 'Away Team']]

In [13]:
data_with_team_ids = pd.merge(data, home_team_name_df, how='left', on='Home Team')
data_with_team_ids = pd.merge(data_with_team_ids, away_team_name_df, how='left', on='Away Team')
data_with_team_ids = data_with_team_ids.rename(columns={'id_x': 'home_team_id', 'id_y': 'away_team_id'})
data_with_team_ids.head()
#list(data_with_team_ids['Home Team'].unique())

,Date,Kick-off (local),Home Team,Away Team,home_score,away_score,Play Off Game?,Home Odds,Draw Odds,Away Odds,home_team_id,away_team_id
0,06-Oct-19,19:30,Sydney Roosters,Canberra Raiders,14,8,Y,\t1.35,\t18.92,\t3.45,15,2
1,28-Sep-19,19:50,Sydney Roosters,Melbourne Storm,14,6,Y,\t1.59,\t19.04,\t2.48,15,7
2,27-Sep-19,19:50,Canberra Raiders,South Sydney Rabbitohs,16,10,Y,\t1.39,\t20.38,\t3.21,2,13
3,21-Sep-19,19:50,Melbourne Storm,Parramatta Eels,32,0,Y,\t1.35,\t21.14,\t3.36,7,11
4,20-Sep-19,19:50,South Sydney Rabbitohs,Manly Warringah Sea Eagles,34,26,Y,\t1.41,\t20.94,\t3.09,13,6


In [14]:
#data = data_with_team_ids
for col in ['Home Odds', 'Away Odds', 'Draw Odds']:
    data_with_team_ids[col] = data_with_team_ids[col].str.replace('\t', '')
    data_with_team_ids[col] = pd.to_numeric(data_with_team_ids[col])
    data_with_team_ids[col] = 1 / data_with_team_ids[col]
data_with_team_ids.head()

,Date,Kick-off (local),Home Team,Away Team,home_score,away_score,Play Off Game?,Home Odds,Draw Odds,Away Odds,home_team_id,away_team_id
0,06-Oct-19,19:30,Sydney Roosters,Canberra Raiders,14,8,Y,0.740741,0.052854,0.289855,15,2
1,28-Sep-19,19:50,Sydney Roosters,Melbourne Storm,14,6,Y,0.628931,0.052521,0.403226,15,7
2,27-Sep-19,19:50,Canberra Raiders,South Sydney Rabbitohs,16,10,Y,0.719424,0.049068,0.311526,2,13
3,21-Sep-19,19:50,Melbourne Storm,Parramatta Eels,32,0,Y,0.740741,0.047304,0.297619,7,11
4,20-Sep-19,19:50,South Sydney Rabbitohs,Manly Warringah Sea Eagles,34,26,Y,0.709220,0.047755,0.323625,13,6


In [15]:
data = data_with_team_ids

In [16]:
data['Date'] = data['Date'] + ' ' + data["Kick-off (local)"]

In [17]:
def convert_to_datetime(date_string):
    try:
        return datetime.strptime(date_string, '%d-%b-%y %H:%M')
    except:
        day = date_string.split(' ')[0]
        return datetime.strptime(day, '%d-%b-%y')
data['Date'] = data['Date'].apply(lambda x: convert_to_datetime(x))

data.dtypes

Date                datetime64[ns]
Kick-off (local)            object
Home Team                   object
Away Team                   object
home_score                  object
away_score                  object
Play Off Game?              object
Home Odds                  float64
Draw Odds                  float64
Away Odds                  float64
home_team_id                 int64
away_team_id                 int64
dtype: object

In [18]:
data = data.drop(columns="Kick-off (local)")
data.head()

,Date,Home Team,Away Team,home_score,away_score,Play Off Game?,Home Odds,Draw Odds,Away Odds,home_team_id,away_team_id
0,2019-10-06 19:30:00,Sydney Roosters,Canberra Raiders,14,8,Y,0.740741,0.052854,0.289855,15,2
1,2019-09-28 19:50:00,Sydney Roosters,Melbourne Storm,14,6,Y,0.628931,0.052521,0.403226,15,7
2,2019-09-27 19:50:00,Canberra Raiders,South Sydney Rabbitohs,16,10,Y,0.719424,0.049068,0.311526,2,13
3,2019-09-21 19:50:00,Melbourne Storm,Parramatta Eels,32,0,Y,0.740741,0.047304,0.297619,7,11
4,2019-09-20 19:50:00,South Sydney Rabbitohs,Manly Warringah Sea Eagles,34,26,Y,0.709220,0.047755,0.323625,13,6


In [19]:
data['Play Off Game?'].value_counts()

Y    99
Name: Play Off Game?, dtype: int64

In [20]:
data['Play Off Game?'] = data['Play Off Game?'].apply(lambda x: True if x == 'Y' else False)

In [21]:
data['Play Off Game?'].value_counts()

False    2112
True       99
Name: Play Off Game?, dtype: int64

In [22]:
data = data[data['Date'].dt.year >= 2013]

In [23]:
data.head()

,Date,Home Team,Away Team,home_score,away_score,Play Off Game?,Home Odds,Draw Odds,Away Odds,home_team_id,away_team_id
0,2019-10-06 19:30:00,Sydney Roosters,Canberra Raiders,14,8,True,0.740741,0.052854,0.289855,15,2
1,2019-09-28 19:50:00,Sydney Roosters,Melbourne Storm,14,6,True,0.628931,0.052521,0.403226,15,7
2,2019-09-27 19:50:00,Canberra Raiders,South Sydney Rabbitohs,16,10,True,0.719424,0.049068,0.311526,2,13
3,2019-09-21 19:50:00,Melbourne Storm,Parramatta Eels,32,0,True,0.740741,0.047304,0.297619,7,11
4,2019-09-20 19:50:00,South Sydney Rabbitohs,Manly Warringah Sea Eagles,34,26,True,0.709220,0.047755,0.323625,13,6


In [24]:
#date, round, home_team_id, home_score, away_team_id, away_score, winner, is_draw, stadium_id, weather, url
data = data.rename(columns={'Date': 'date',
                            'Play Off Game?': 'is_playoff',
                            'Home Odds': 'home_odds',
                            'Draw Odds': 'draw_odds',
                            'Away Odds': 'away_odds'})
data.head()

,date,Home Team,Away Team,home_score,away_score,is_playoff,home_odds,draw_odds,away_odds,home_team_id,away_team_id
0,2019-10-06 19:30:00,Sydney Roosters,Canberra Raiders,14,8,True,0.740741,0.052854,0.289855,15,2
1,2019-09-28 19:50:00,Sydney Roosters,Melbourne Storm,14,6,True,0.628931,0.052521,0.403226,15,7
2,2019-09-27 19:50:00,Canberra Raiders,South Sydney Rabbitohs,16,10,True,0.719424,0.049068,0.311526,2,13
3,2019-09-21 19:50:00,Melbourne Storm,Parramatta Eels,32,0,True,0.740741,0.047304,0.297619,7,11
4,2019-09-20 19:50:00,South Sydney Rabbitohs,Manly Warringah Sea Eagles,34,26,True,0.709220,0.047755,0.323625,13,6


In [25]:
data['winner'] = 0
data['is_draw'] = False
for col in ['winner', 'is_draw', 'home_score', 'away_score']:
    data[col] = data[col].astype(int)
#data.head()
def update_winner(match):
    if match['home_score'] > match['away_score']:
        return match['home_team_id']
    elif match['home_score'] < match['away_score']:
        return match['away_team_id']
    else:
        return 0
    
def find_draws(match):
    if match['winner'] == 0:
        return True
    else:
        return False

data['winner'] = data.apply(lambda x: update_winner(x), axis = 1)
data['is_draw'] = data.apply(lambda x: find_draws(x), axis = 1)
# data.head()

In [26]:
data.tail(15)

,date,Home Team,Away Team,home_score,away_score,is_playoff,home_odds,draw_odds,away_odds,home_team_id,away_team_id,winner,is_draw
1392,2013-03-17 18:30:00,Manly Warringah Sea Eagles,Newcastle Knights,32,0,False,0.591716,0.053107,0.450450,6,8,6,False
1393,2013-03-17 15:00:00,Wests Tigers,Penrith Panthers,28,18,False,0.595238,0.051467,0.452489,16,12,16,False
1394,2013-03-17 13:00:00,Gold Coast Titans,Canberra Raiders,36,0,False,0.699301,0.044111,0.353357,5,2,5,False
1395,2013-03-16 18:30:00,North Queensland Cowboys,Melbourne Storm,10,32,False,0.537634,0.056465,0.505051,10,7,7,False
1396,2013-03-16 19:30:00,New Zealand Warriors,Sydney Roosters,14,16,False,0.450450,0.053022,0.595238,9,15,15,False
1397,2013-03-15 20:05:00,St. George Illawarra Dragons,Brisbane Broncos,6,22,False,0.467290,0.055556,0.578035,14,1,1,False
1398,2013-03-14 20:05:00,Parramatta Eels,Canterbury-Bankstown Bulldogs,16,20,False,0.438596,0.052247,0.609756,11,3,3,False
1399,2013-03-11 19:00:00,Newcastle Knights,Wests Tigers,42,10,False,0.680272,0.052632,0.371747,8,16,8,False
1400,2013-03-10 18:30:00,Cronulla-Sutherland Sharks,Gold Coast Titans,12,10,False,0.649351,0.045455,0.381679,4,5,4,False
1401,2013-03-10 15:00:00,Melbourne Storm,St. George Illawarra Dragons,30,10,False,0.813008,0.045809,0.238095,7,14,7,False


## Adding Rounds to data

In [27]:
import json
bye_weeks = pd.read_json('bye_weeks.json')
bye_weeks['items']

0    {'season': 2013, 'rounds': 26, 'exceptions': {...
1    {'season': 2014, 'rounds': 26, 'exceptions': {...
2    {'season': 2015, 'rounds': 26, 'exceptions': {...
3    {'season': 2016, 'rounds': 26, 'exceptions': {...
4    {'season': 2017, 'rounds': 26, 'exceptions': {...
5    {'season': 2018, 'rounds': 25, 'exceptions': {...
6    {'season': 2019, 'rounds': 25, 'exceptions': {...
7    {'season': 2020, 'rounds': 25, 'exceptions': {...
Name: items, dtype: object

In [28]:
matches_by_year = dict.fromkeys(list(data['date'].dt.year.unique()), [])
for year in matches_by_year.keys():
    matches_by_year[year] = data[data['date'].dt.year == year].sort_values(by='date').reset_index()
matches_by_year[2013]

,index,date,Home Team,Away Team,home_score,away_score,is_playoff,home_odds,draw_odds,away_odds,home_team_id,away_team_id,winner,is_draw
0,1406,2013-03-07 20:05:00,Sydney Roosters,South Sydney Rabbitohs,10,28,False,0.429185,0.055036,0.621118,15,13,13,False
1,1405,2013-03-08 19:05:00,Brisbane Broncos,Manly Warringah Sea Eagles,14,22,False,0.632911,0.057703,0.409836,1,6,6,False
2,1404,2013-03-09 17:30:00,Parramatta Eels,New Zealand Warriors,40,10,False,0.561798,0.056085,0.483092,11,9,11,False
3,1403,2013-03-09 19:30:00,Canterbury-Bankstown Bulldogs,North Queensland Cowboys,12,24,False,0.478469,0.056593,0.561798,3,10,10,False
4,1402,2013-03-10 14:00:00,Penrith Panthers,Canberra Raiders,32,10,False,0.515464,0.057143,0.526316,12,2,12,False
5,1401,2013-03-10 15:00:00,Melbourne Storm,St. George Illawarra Dragons,30,10,False,0.813008,0.045809,0.238095,7,14,7,False
6,1400,2013-03-10 18:30:00,Cronulla-Sutherland Sharks,Gold Coast Titans,12,10,False,0.649351,0.045455,0.381679,4,5,4,False
7,1399,2013-03-11 19:00:00,Newcastle Knights,Wests Tigers,42,10,False,0.680272,0.052632,0.371747,8,16,8,False
8,1398,2013-03-14 20:05:00,Parramatta Eels,Canterbury-Bankstown Bulldogs,16,20,False,0.438596,0.052247,0.609756,11,3,3,False
9,1397,2013-03-15 20:05:00,St. George Illawarra Dragons,Brisbane Broncos,6,22,False,0.467290,0.055556,0.578035,14,1,1,False


In [29]:
#Remove JSON entry for 2020 as no matches have been played yet
bye_weeks['items'].pop(7)

{'season': 2020, 'rounds': 25, 'exceptions': {'12': 4, '16': 4}}

In [30]:
match_round_columns = dict.fromkeys(list(data['date'].dt.year.unique()), [])
for item in bye_weeks['items']:
    season = item['season']
    rounds = item['rounds']
    exception_dict = item['exceptions']
    exception_rounds = list(item['exceptions'].keys())
    exception_matches = list(item['exceptions'].values())
    
    round_list = []
    for i in range(1, rounds+1):
        if str(i) not in exception_rounds:
            for each in range(1,9):
                round_list.append(i)
        else:
            range_ending = exception_dict[str(i)] + 1
            for each in range(1, range_ending):
                round_list.append(i)
    for i in range(1,5):
        round_list.append('finals-week-1')
    for i in range(1,3):
        round_list.append('finals-week-2')
    for i in range(1,3):
        round_list.append('finals-week-3')
    round_list.append('grand-final')
    match_round_columns[season] = round_list
    
print(match_round_columns)

{2019: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 'finals-week-1', 'finals-week-1', 'finals-week-1', 'finals-week-1', 'finals-week-2', 'finals-week-2', 'finals-week-3', 'finals-week-3', 'grand-final'], 2018: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,

In [31]:
for year in matches_by_year.keys():
    #df = matches_by_year[year]
    round_column = pd.Series(match_round_columns[year])
    matches_by_year[year]['round'] = round_column

In [32]:
def format_team_name(team):
    #print(team.split(' ')[-1].lower().strip())
    if team == 'Manly Warringah Sea Eagles':
        team = 'sea-eagles'
    elif team == 'Wests Tigers':
        team = 'wests-tigers'
    else:
        team = team.split(' ')[-1].lower().strip()
    return team
    
def create_url_column(match):
    home_team = format_team_name(match['Home Team'])
    away_team = format_team_name(match['Away Team'])
    year = str(match['date'].year)
    #print(match.iloc)
    try:
        match_round = 'round-' + str(int(match['round']))
    except:
        match_round = match['round']
    if year == '2019':
        vs = '-v-'
    else:
        vs = '-vs-'
    
    url = 'https://www.nrl.com/draw/nrl-premiership/' + year + '/' + match_round + '/' + home_team + vs + away_team + '/'
    return url

In [33]:
for year in matches_by_year.keys():
    year_df = matches_by_year[year]
    year_df['url'] = year_df.apply(lambda x: create_url_column(x), axis = 1)
    
matches_by_year[2018]
#iloc 192 - 195 = week 1
#iloc 196 - 197 = week 2
#iloc 198 - 199 = week 3
#iloc 200 = grand final

,index,date,Home Team,Away Team,home_score,away_score,is_playoff,home_odds,draw_odds,away_odds,home_team_id,away_team_id,winner,is_draw,round,url
0,401,2018-03-08 19:50:00,St. George Illawarra Dragons,Brisbane Broncos,34,12,False,0.588235,0.048757,0.438596,14,1,14,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
1,400,2018-03-09 18:00:00,Newcastle Knights,Manly Warringah Sea Eagles,19,18,False,0.518135,0.050125,0.512821,8,6,8,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
2,399,2018-03-09 18:50:00,North Queensland Cowboys,Cronulla-Sutherland Sharks,20,14,False,0.598802,0.048170,0.432900,10,4,10,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
3,397,2018-03-10 16:00:00,South Sydney Rabbitohs,New Zealand Warriors,20,32,False,0.680272,0.047551,0.350877,13,9,9,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
4,398,2018-03-10 16:30:00,Wests Tigers,Sydney Roosters,10,8,False,0.250000,0.036887,0.787402,16,15,16,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
5,396,2018-03-10 18:00:00,Canterbury-Bankstown Bulldogs,Melbourne Storm,18,36,False,0.318471,0.045269,0.714286,3,7,7,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
6,395,2018-03-11 16:10:00,Penrith Panthers,Parramatta Eels,24,14,False,0.507614,0.049407,0.523560,12,11,12,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
7,394,2018-03-11 17:30:00,Gold Coast Titans,Canberra Raiders,30,28,False,0.432900,0.048263,0.598802,5,2,5,False,1,https://www.nrl.com/draw/nrl-premiership/2018/...
8,393,2018-03-15 19:50:00,Cronulla-Sutherland Sharks,St. George Illawarra Dragons,16,20,False,0.510204,0.049188,0.518135,4,14,14,False,2,https://www.nrl.com/draw/nrl-premiership/2018/...
9,392,2018-03-16 18:00:00,Sydney Roosters,Canterbury-Bankstown Bulldogs,30,12,False,0.751880,0.042662,0.289855,15,3,15,False,2,https://www.nrl.com/draw/nrl-premiership/2018/...


In [45]:
find_match_id_sql ='''
    SELECT id 
    FROM Matches
    WHERE home_team_id = %s
        AND away_team_id = %s
        AND DATE(date) = %s;
    '''
insert_sql = '''
            INSERT INTO Matches (date, round, home_team_id, home_score, away_team_id, away_score, 
                winner, is_draw, url, home_odds, draw_odds, away_odds, is_playoff)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            '''
update_sql = '''
            UPDATE Matches SET home_odds = %s, 
                draw_odds = %s,
                away_odds = %s,
                is_playoff = False
            WHERE id = %s;
            '''

In [48]:
match_df = pd.concat(matches_by_year.values(), ignore_index=True)
for col in ['home_odds', 'draw_odds', 'away_odds']:
    match_df[col] = match_df[col].round(4)
match_df

,index,date,Home Team,Away Team,home_score,away_score,is_playoff,home_odds,draw_odds,away_odds,home_team_id,away_team_id,winner,is_draw,round,url
0,200,2019-03-14 19:50:00,Melbourne Storm,Brisbane Broncos,22,12,False,0.6173,0.0488,0.4065,7,1,7,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
1,199,2019-03-15 18:00:00,Newcastle Knights,Cronulla-Sutherland Sharks,14,8,False,0.5525,0.0541,0.4854,8,4,8,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
2,198,2019-03-15 19:55:00,Sydney Roosters,South Sydney Rabbitohs,16,26,False,0.6494,0.0506,0.3922,15,13,13,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
3,197,2019-03-16 17:00:00,New Zealand Warriors,Canterbury-Bankstown Bulldogs,40,6,False,0.6061,0.0509,0.4292,9,3,9,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
4,196,2019-03-16 17:30:00,Wests Tigers,Manly Warringah Sea Eagles,20,6,False,0.6135,0.0537,0.4202,16,6,16,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
5,195,2019-03-16 18:35:00,North Queensland Cowboys,St. George Illawarra Dragons,24,12,False,0.5495,0.0554,0.4878,10,14,10,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
6,194,2019-03-17 16:05:00,Penrith Panthers,Parramatta Eels,12,20,False,0.6757,0.0502,0.3636,12,11,11,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
7,193,2019-03-17 17:10:00,Gold Coast Titans,Canberra Raiders,0,21,False,0.5291,0.0557,0.5128,5,2,2,False,1,https://www.nrl.com/draw/nrl-premiership/2019/...
8,192,2019-03-21 19:50:00,St. George Illawarra Dragons,South Sydney Rabbitohs,18,34,False,0.4098,0.0531,0.6173,14,13,13,False,2,https://www.nrl.com/draw/nrl-premiership/2019/...
9,191,2019-03-22 18:00:00,Canberra Raiders,Melbourne Storm,10,22,False,0.4464,0.0503,0.5917,2,7,7,False,2,https://www.nrl.com/draw/nrl-premiership/2019/...


In [51]:
for match in match_df.iterrows():
    match = match[1]
    if match['is_playoff'] == True:
        match['round'] = None
    if (match['date'].year in [2018, 2019]) & (not match['is_playoff']):
        mycursor.execute(find_match_id_sql, (match['home_team_id'], match['away_team_id'], match['date'].date()))
        match_id = mycursor.fetchone()[0]
        #print(match['home_odds'], match['draw_odds'], match['away_odds'], match_id)
        #print('update_sql')
        mycursor.execute(update_sql, (match['home_odds'], match['draw_odds'], match['away_odds'], match_id))
        mydb.commit()
    else:
        #print('\n')
        #print(match)
        #print(match['date'], match['round'], match['home_team_id'], match['home_score'],
        #                             match['away_team_id'], match['away_score'], match['winner'], match['is_draw'],
        #                             match['url'], match['home_odds'], match['draw_odds'], match['away_odds'], match['is_playoff'])
        mycursor.execute(insert_sql, (match['date'], match['round'], match['home_team_id'], match['home_score'],
                                     match['away_team_id'], match['away_score'], match['winner'], match['is_draw'],
                                     match['url'], match['home_odds'], match['draw_odds'], match['away_odds'], match['is_playoff']))
        mydb.commit()